# <font color='red'>PRE-CONDITION</font>: Have a programmatic user account in AWS, fill in dwh.cfg file with its Key and Secret:
<font color='green'>[AWS_MANAGER]</font><br>
KEY=<font color='red'>HERE</font> <br>
SECRET=<font color='red'>HERE</font>

# 1. Load AWS DWH Params from dwh.cfg file

In [ ]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS_MANAGER','KEY')
SECRET                 = config.get('AWS_MANAGER','SECRET')

DWH_CLUSTER_TYPE       = config.get("RS_DWH_CLUSTER_CREATION_INFO","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("RS_DWH_CLUSTER_CREATION_INFO","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("RS_DWH_CLUSTER_CREATION_INFO","DWH_NODE_TYPE")
DWH_CLUSTER_IDENTIFIER = config.get("RS_DWH_CLUSTER_CREATION_INFO","DWH_CLUSTER_IDENTIFIER")

DB_NAME                = config.get("CLUSTER","DB_NAME")
DB_USER                = config.get("CLUSTER","DB_USER")
DB_PASSWORD            = config.get("CLUSTER","DB_PASSWORD")
DB_PORT                = config.get("CLUSTER","DB_PORT")

DWH_IAM_ROLE_NAME      = config.get("IAM_ROLE_CREATION_INFO", "DWH_IAM_ROLE_NAME")

# 2. Create AWS IAM and Redshift clients

In [ ]:
import boto3

iam_client = boto3.client('iam', region_name='us-west-2', aws_access_key_id=KEY, aws_secret_access_key=SECRET)
rs_client = boto3.client('redshift', region_name='us-west-2', aws_access_key_id=KEY, aws_secret_access_key=SECRET)

# 3. Create an IAM Role that will make Redshift able to access S3 bucket (ReadOnly)

In [ ]:
import json
try:
    dwhRole = iam_client.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description="Allows redshift cluster to call AWS services on your behalf",
        AssumeRolePolicyDocument=json.dumps(
            {
                'Statement': [
                    {
                        'Action':'sts:AssumeRole',
                        'Effect': 'Allow',
                        'Principal': {
                            'Service': 'redshift.amazonaws.com'
                        }
                    }
                ],
                'Version': '2012-10-17'
            }
        )
    )
except Exception as e:
    print(e)

In [ ]:
try:
    iam_client.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                           PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                           )['ResponseMetadata']['HTTPStatusCode']
except Exception as e:
    print(e)

In [ ]:
roleArn = iam_client.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

# 4. Create Redshift (RS) Cluster

In [ ]:
try:
    response = rs_client.create_cluster(        
        # parameters for hardware
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        # identifiers & credentials
        DBName=DB_NAME,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DB_USER,
        MasterUserPassword=DB_PASSWORD,
        
        # role (to allow s3 access)
        IamRoles=[roleArn] 
    )
except Exception as e:
    print(e)

### 4.1. <font color='red'> Check cluster status executing below line as many times needed. Move to the next commands only when the cluster status becomes </font> <font color='green'> available</font>

In [ ]:
import pandas as pd

def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = rs_client.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

## 5. Take note of the Cluster endpoint and ARN 
## <font color='red'>===> Copy below information to *dwh.cfg* file</font>

<font color='green'>[CLUSTER]</font><br>
HOST=<font color='red'>HERE</font> <br>
<font color='green'>[IAM]</font><br>
ARN=<font color='red'>HERE</font>


In [ ]:
HOST = myClusterProps['Endpoint']['Address']
ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("HOST :: ", HOST)
print("ARN :: ", ARN)

# 6. AWS RESOURCES COSTS MONEY, CLEAN IT UP AT THE END OF YOUR PROJECT EXECUTION!!!!!!

### Delete Redshift created cluster

In [ ]:
rs_client.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)

#### Don't leave this notebook while the ClusterStatus is <font color='red'> deleting </font>, wait for an error message.

In [ ]:
import pandas as pd

def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = rs_client.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

### Detach and delete created IAM role

In [ ]:
iam_client.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam_client.delete_role(RoleName=DWH_IAM_ROLE_NAME)